# Data Structures: `Map` and `Timeseries`

In [ ]:
import copy
import glob

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
from IPython.display import HTML

import astropy.table
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.visualization import AsymmetricPercentileInterval, ImageNormalize, LogStretch

import sunpy.map
import sunpy.timeseries
import sunpy.sun.constants
from sunpy.coordinates import Helioprojective

## The `Map` Data Structure

In [ ]:
aia_files = sorted(glob.glob('data/AIA/*.fits'))
stereo_files = sorted(glob.glob('data/SECCHI/*.fts'))
eui_files = sorted(glob.glob('data/EUI/*.fits'))

We create a `sunpy.map.Map` object by passing in the FITS file for a single AIA, SECCHI (STEREO), and EUI observation.
We've purposefully chosen an observation after the eruption started.

In [ ]:
m_aia = sunpy.map.Map(aia_files[6])
m_stereo = sunpy.map.Map(stereo_files[6])
m_eui = sunpy.map.Map(eui_files[2])

We can easily visualize a map after loading it using the quicklook functionality.

In [ ]:
m_stereo

In [ ]:
m_stereo.plot()

We will talk much more about the `plot` command in later sections.

### Attributes of `Map`

`Map` provides a common interface to most 2D imaging solar datasets and provides several useful pieces of metadata.
As mentioned in the intro slide, `Map` is a container for holding your data and metadata (usually from the FITS header) together.

The `.meta` and `.data` attributes provide access to the metadata and underlying array of image data, respectively.

In [ ]:
m_aia.data

In [ ]:
m_aia.meta

However, this metadata can be terse, non-homogeneous, and sometimes difficult to parse.
`Map` provides several attributes derived from the underlying raw metadata that expose a uniform interface to the metadata for each map.

In [ ]:
m_aia.wavelength

In [ ]:
m_aia.rsun_meters

In [ ]:
m_aia.rsun_obs

In [ ]:
m_eui.rsun_obs

In [ ]:
m_eui.processing_level

In [ ]:
m_stereo.instrument

Each `Map` object also holds the unit system that the image data is in, expressed in terms of an `astropy.unit.Unit` object.

In [ ]:
m_eui.unit

### Coordinate Information

Each `Map` also exposes information about which coordinate system the image was taken in, including the location of the spacecraft that recorded that observation.

`sunpy` leverages and extends the powerful `astropy` coordinate framework that we heard about in the previous tutorial.
Additionally, we'll talk more about the `sunpy.coordinates` subpackage in the next notebook and show some neat examples.

For each `Map`, we can easily access what *coordinate frame* the observation cooresponds to.

In [ ]:
m_aia.coordinate_frame

Similarly, we can look at the location of the observer (as defined by the position of the satellite at the time of the observation).

In [ ]:
m_aia.observer_coordinate

In [ ]:
m_stereo.observer_coordinate

In [ ]:
m_eui.observer_coordinate

We can plot these observer coordinates to show the relative position, in heliographic longitude, of each spacecraft, similar to the SolarMACH plot we showed in our previous notebook.

(**NOTE:** *It is not particuarly important to understand the intricacies of the plotting code below. This is merely to show we can use the coordination information in each map to visualize the relative positions of the three spacecraft we are concerned with here.*)

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = plt.subplot(projection='polar')

# Plot the Sun
ax.plot(0, 0, marker='o', markersize=20, label='Sun', color='yellow')

# Plot the satellite locations
for m in [m_aia, m_stereo, m_eui]:
    sat = m.observatory
    coord = m.observer_coordinate
    ax.plot(coord.lon.to('rad'), coord.radius.to(u.AU), 'o', label=sat)

ax.set_theta_zero_location("S")
ax.set_rlabel_position(90)
ax.set_rlim(0, 1.3)
ax.legend()

### Visualization

In the astropy tutorial, we learned about how to create axes with projections derived from a particular WCS.
`Map` provides some additional "helpers" for plotting the associated image data with the correct projection based on the WCS.

At a minimum, this can be accomplished through the `.plot()` method.

In [ ]:
m_aia.plot()

This "automagically" creates a figure and an axis (with a projection based on the WCS of the map) and plots our map on that axis, with a colormap and normalization tailored for the specific map source.
All of this visualization is built on top of `matplotlib` and the `WCSAxes` capabilities provided by `astropy` that we saw demoed earlier today.
However, as you can see, the resulting default colorbar is not particularly useful.

Because all of this plotting capability is built on top of `matplotlib`, we can easily customize the various components of our plot.

In [ ]:
plt.figure(figsize=(8,8))
m_aia.plot(vmin=0,vmax=5e2)
m_aia.draw_grid(lw=1, alpha=1)

We can explictly create the axis and add the projection for the map if we want more fine-grained control over labels and titles.
We can also easily adjust the limits on our colorbar using the `clip_interval` key.

In [ ]:
fig = plt.figure(figsize=(6,5))
ax = fig.add_subplot(111, projection=m_aia)
im = m_aia.plot(axes=ax, clip_interval=(25,99.6)*u.percent)
ax.set_title(r'A nicer AIA 304 $\mathrm{\AA}$ Plot')
ax.coords[0].set_axislabel('HPC Lon')
ax.coords[1].set_axislabel('HPC Lat')
fig.colorbar(im)

Or specify a new normalization altogether.

In [ ]:
norm = ImageNormalize(vmin=0, vmax=200, stretch=LogStretch())

In [ ]:
plt.figure(figsize=(8,8))
m_eui.plot(norm=norm)

<div class="alert alert-block alert-warning">
    <h3><u>EXERCISE:</u> <br><br>How would I change the colormap for the above plot?</h3>
</div>

In [ ]:
# INSTRUCTOR BLOCK
plt.figure(figsize=(8,8))
m_eui.plot(norm=norm, cmap='inferno')

In [ ]:
# STUDENT BLOCK

Using `matplotlib` combined with `WCSAxes`, we can build more complex, publication-quality visualizations.

(**NOTE:** It is not necessary to fully understand every intricacy of the plotting code below during the course of the tutorial. This is merely to show how `Map.plot` can be be used to make more complex plots.)

In [ ]:
fig = plt.figure(figsize=(15,5))

for i, m in enumerate([m_aia, m_stereo, m_eui]):
    # Create the axis with the appropriate projection
    ax = fig.add_subplot(1,3,i+1,projection=m)
    
    # Add the plot to the axis
    im = m.plot(axes=ax, annotate=False, clip_interval=(25,99.5)*u.percent)
    
    # Make the HPC grid lines visible
    ax.coords.grid(alpha=1, ls='-')
    
    # Adjust the labels and ticks
    if i > 0:
        ax.coords[1].set_auto_axislabel(False)
    else:
        ax.coords[1].set_axislabel('Solar-Y')
    ax.coords[0].set_axislabel('Solar-X')
    ax.coords[1].set_ticklabel(rotation=90,)
    
    # Put a label on each plot
    ax.text(m.data.shape[1]//2, m.data.shape[0]*.97, m.observatory,
            color='w',
            horizontalalignment='center',
            verticalalignment='top',
            fontsize=14)
    
    # Add a colorbar to the top of each plot
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('top', size='4%', pad=0.1, axes_class=matplotlib.axes.Axes)
    fig.colorbar(im, cax=cax, orientation='horizontal')
    cax.xaxis.set_ticks_position("top")
    cax.xaxis.set_tick_params(direction='in')
    
plt.subplots_adjust(wspace=0.1)

<div class="alert alert-block alert-warning">
    <h3><u>EXERCISE:</u> <br><br>Do you know why the grid lines in the STEREO and EUI images appear tilted relative to the image?</h3>
</div>

### Animations with `MapSequence`

In addition, the `MapSequence` container provides a data container for holding multiple maps, such as in the case where you have a sequence of maps taken at successive times.
We can create `MapSequence` objects by passing in our list of files and the `sequence=True` keyword argument.

In [ ]:
eui_seq = sunpy.map.Map(eui_files, sequence=True)

The `MapSequence` can be indexed to return the individual `Map` objects at each time step.

In [ ]:
eui_seq[0]

One of the most useful features of a `MapSequence` is the ability to create coordinate-aware visualizations of our stack of `Map` objects.
To do this, we'll first create a a colormap normalization appropriate to the range of the data for every map in our stack.

In [ ]:
vmin, vmax = AsymmetricPercentileInterval(1,99.5).get_limits(eui_seq.as_array())
norm=ImageNormalize(vmin=vmin, vmax=vmax, stretch=LogStretch())

The `plot` method on our `MapSequence` object now returns an animation rather than a simple static plot.

In [ ]:
plt.figure(figsize=(10,10))
eui_ani = eui_seq.plot(norm=norm)

In [ ]:
HTML(eui_ani.to_jshtml())

We can do this with the STEREO data as well.

In [ ]:
stereo_seq = sunpy.map.Map(stereo_files, sequence=True)
vmin, vmax = AsymmetricPercentileInterval(1,99.5).get_limits(stereo_seq.as_array())
norm=ImageNormalize(vmin=vmin, vmax=vmax, stretch=LogStretch())
plt.figure(figsize=(10,10))
stereo_ani = stereo_seq.plot(norm=norm)

In [ ]:
HTML(stereo_ani.to_jshtml())

### `Map` and WCS

As we saw in the `astropy` tutorial, the world coordinate system (WCS) formalizes provides us a framework for transforming between pixel and world coordinates.

In [ ]:
m_aia.wcs

In [ ]:
type(m_aia.wcs)

We can use the associated `pixel_to_world` and `world_to_pixel` functions to transform between the world and pixel coordinates of our images.

In [ ]:
m_aia.wcs.pixel_to_world(0*u.pix, 0*u.pix)

In [ ]:
m_aia.bottom_left_coord

The `bottom_left_coord` is the *center* of the pixel in the bottom left corner of our image.

In [ ]:
m_aia.wcs.world_to_pixel(m_aia.bottom_left_coord)

Similarly, we can confirm that the `center` coordinate falls on the center of the map.

In [ ]:
m_aia.center

In [ ]:
m_aia.wcs.world_to_pixel(m_aia.center)

In [ ]:
m_aia.dimensions

Note that the center of our AIA image does not align with the center of the Sun!

In [ ]:
m_aia.wcs.world_to_pixel(SkyCoord(Tx=0*u.arcsec, Ty=0*u.arcsec, frame=m_aia.coordinate_frame))

<div class="alert alert-block alert-warning">
    <h3><u>EXERCISE:</u> <br><br>How would you find the position of the center of the EUI map in the pixel coordinates of the AIA map?</h3>
</div>

### Basic Image Manipulation

There are several methods on the `Map` object that provide capabilities for doing basic image manipulation in combination with the coordinate information attached to each `Map`.

#### Rotate

The `.rotate` method applies a rotation in the image plane, i.e. about an axis out of the page. 
In the case where we do not specify an angle (or rotation matrix), the image will be rotated such that the world and pixel axes are aligned.
In the case of an image in helioprojective coordinate system, this means that solar north will be aligned with the y-like pixel axis of the image.

In [ ]:
m_stereo_rot = m_stereo.rotate(missing=m_stereo.min())

By default, any missing values will be filled with "NaN". Here, we specify `missing` as the minimum intensity value of the map such.

In [ ]:
fig = plt.figure(figsize=(11,5))
ax = fig.add_subplot(121,projection=m_stereo)
m_stereo.plot(axes=ax, vmin=800, vmax=5e3)
ax.coords.grid(alpha=1, ls='-')
ax = fig.add_subplot(122,projection=m_stereo_rot)
m_stereo_rot.plot(axes=ax, vmin=800, vmax=5e3)
ax.coords.grid(alpha=1, ls='-')

This rotation is also reflected in the updated metadata of the rotated image.

In [ ]:
m_stereo.rotation_matrix

In [ ]:
m_stereo_rot.rotation_matrix

Additionally, one can also specify some arbitrary angle to rotate the image by.
Note that this angle is relative to the current orientation of the image.

In [ ]:
m_stereo_rot30 = m_stereo.rotate(angle=30*u.deg, missing=m_stereo.min())

In [ ]:
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111,projection=m_stereo_rot30)
m_stereo_rot30.plot(axes=ax)
ax.coords.grid(alpha=1, ls='-')

<div class="alert alert-block alert-warning">
    <h3><u>EXERCISE:</u> <br><br>How would you rotate the image such that there is exactly a 45 degree orientation between the world and pixel axes?</h3>
</div>

We'll apply a rotation to the AIA map as well.

In [ ]:
m_aia_rot = m_aia.rotate(missing=m_aia.min())

#### Cropping Images with `submap`

We commonly want to pare down our full field-of-view to a particular region of interest.
With a map, we can do this using the `submap` method.

We can specify the region of our submap using world coordinates as specified by a `SkyCoord`.
We will specify these coordinates in Heliographic Stonyhurst (HGS) coordinates.
From the animation of the STEREO data above, we can identify approximately where the CME was launched from and crop our image around that region.

In [ ]:
bottom_left = SkyCoord(lon=-20*u.deg, lat=-5*u.deg, radius=1*sunpy.sun.constants.radius,
                       frame='heliographic_stonyhurst', obstime=m_aia.date)
top_right = SkyCoord(lon=30*u.deg, lat=35*u.deg, radius=1*sunpy.sun.constants.radius,
                     frame='heliographic_stonyhurst', obstime=m_aia.date)

In [ ]:
m_stereo_cropped = m_stereo.submap(bottom_left, top_right=top_right)

In [ ]:
m_stereo_cropped

<div class="alert alert-block alert-warning">
    <h3><u>EXERCISE:</u> <br><br>The coordinates for our cutout can also be specified in pixel coordinates. Find the corners of our cutout in pixel coordinates and then create the same submap using those pixel coordinates.
</h3>
</div>

We can use those exact same coordinates to create a cutout from our AIA map.

In [ ]:
m_aia_cropped = m_aia.submap(bottom_left, top_right=top_right)

In [ ]:
m_aia_cropped

## The `Timeseries` Data Structure

In addition to `Map` for 2D image data, `sunpy` also provides a container for tabular time series data through the `TimeSeries` class.
We can create a `TimeSeries` object in a very similar manner to how we create a `Map` object.

Let's look at the corresponding GOES XRS data that we downloaded in the previous notebook.

In [ ]:
goes_files = sorted(glob.glob('data/XRS/*.nc'))

In [ ]:
ts = sunpy.timeseries.TimeSeries(goes_files)

In [ ]:
ts

As with `Map`, `TimeSeries` acts as a container for the data + metadata. We can access each component individually.

In [ ]:
ts.meta

The `TimeSeries` object can also be converted to other formats like an `astropy` `Table` object

In [ ]:
ts.to_table()

or a `pandas` `DataFrame`

In [ ]:
ts.to_dataframe()

There are also a number of attributes on each `TimeSeries` derived from the data/metadata.

In [ ]:
ts.columns

In [ ]:
ts.observatory

In [ ]:
ts.source

In [ ]:
ts.units

### Slicing and Visualizing `TimeSeries`

Note that this intensity `TimeSeries` spans 24 h of observation time and recall that we are only interested in the ~3 h interval in which the CME is visible in the 304 channel.
We can truncate our timeseries around the times of interest.
To do this, we can actually use the `date` property on our first and last EUI map from our sequence.

In [ ]:
ts_cme = ts.truncate(eui_seq[0].date.iso, eui_seq[-1].date.iso)

And then do a quicklook on our lightcurve.

In [ ]:
ts_cme

As expected, we find that there is a flare occuring right around the time the CME occurs. This should not be surprising as we saw from the AIA data that the CME was Earth-directed such that GOES was well-position to observed the flare.

We can also zoom in a bit on the beginning of the flare.

In [ ]:
fig = plt.figure()
ax = fig.gca()
ts_cme.plot(axes=ax)
ax.set_xlim('2022-03-28 11:00', '2022-03-28 11:30')

As expected, we find that the flare, as detected by GOES, begins just before the eruption is seen the 304 channel of EUI at 11:20.